# Importing Packages

In [ ]:
# importing packages
from itertools import permutations, count
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
from sklearn.model_selection import (train_test_split,
                                     cross_val_score,
                                     cross_validate,
                                     KFold)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.compose import TransformedTargetRegressor
import optuna
from sklearn.metrics import *
from sklearn.inspection import *
import shap
import lovelyplots
import statsmodels.api as sm
import gc
import warnings


import scienceplots


import sklearn
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from scipy import stats

import tensorflow as tf
import keras_tuner


import missingno


# Plotting Configuration

In [2]:
plt.rcdefaults()
mpl_global_config = {
    'figure.figsize': (7, 7),
    'figure.dpi': 1000,
    'font.size': 16,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 8,
    'lines.linewidth': 2,
    'lines.markersize': 3,
    'grid.linewidth': 0.75,
    'savefig.dpi': 1000,
    'savefig.transparent': False,
    'savefig.bbox': 'tight',
    'pdf.compression': 9,
    'axes.axisbelow': True
}
plt.rcParams.update(mpl_global_config)
plt.style.use(['science', 'nature', 'high-contrast', "no-latex"])


colors = {
    "yellow": "#DDAA33",
    "red": "#BB5566",
    "blue": "#004488",
    "black": "#000000",
    "white": "#FFFFFF"
}

# Global Configuration Setting Controling Randomness, Trials, etc

In [3]:
# config initialization
set_config(transform_output="pandas")
np.seterr(under='ignore')
warnings.filterwarnings('ignore')
SEED = 0
n_trials = 75
%matplotlib inline

# def evaluation

In [4]:
def evaluation(trained_model, X, y_real):

    y_pred = trained_model.predict(X)

    mse = mean_squared_error(y_real, y_pred, squared=True)
    rmse = mean_squared_error(y_real, y_pred, squared=False)
    mae = mean_absolute_error(y_real, y_pred)
    r2 = r2_score(y_real, y_pred)
    maxerror = max_error(y_real, y_pred)
    mape = mean_absolute_percentage_error(y_real, y_pred)

    return {"MeanSquaredError":mse,
            "RootMeanSquaredError": rmse,
            "MeanAbsoluteError": mae,
            "R2": r2,
            "MaxError": maxerror,
            "MAPE": mape}

# Read the Preprocessed Data

In [5]:
# reading the dataset
df = pd.read_csv("df.csv",
                 dayfirst=True,
                 parse_dates=True,
                 index_col="Date")

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [13]:
data_summary = df.describe()
data_summary.loc["Missing Values Ratio"] = df.isnull().sum().divide(len(df))
data_summary.loc["Unique Values Ratio"] = df.nunique().divide(len(df))
data_summary.loc["Kurtosis"] = df.kurtosis()
data_summary.loc["Skewness"] = df.skew()
data_summary.T.to_csv("data_summary.csv")

In [ ]:
data_summary.T

# Separating X and y

In [ ]:
X = df.drop(columns=["Total Biogas Flowrate (m3/d)"])
y = df.loc[: , "Total Biogas Flowrate (m3/d)"]
print(f"dataframe shape: {df.shape}\n"\
      f"features shape: {X.shape}\n"\
      f"target shape: {y.shape}")

# Train and Test Split and Normalization

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=SEED
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Hyperparameter Optimization

In [ ]:
# train a Decision Tree model with tpe sampler and median pruner
def objective_dt(trial):

    dt_params = {
            "max_depth": trial.suggest_int("max_depth", 2, 15),
            "min_samples_leaf": trial.suggest_float("min_samples_leaf", 0.01, 0.5),
        }

    model = DecisionTreeRegressor(random_state=SEED, **dt_params)

    score = -np.mean(
        cross_val_score(
                model,
                X_train,
                y_train,
                cv=KFold(n_splits=5),
                n_jobs=-1,
                scoring='neg_mean_squared_error')
        )

    return score

study_dt = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.HyperbandPruner()
)

study_dt.optimize(objective_dt, n_trials=n_trials)

In [ ]:
# train a Random Forest model with tpe sampler and median pruner
def objective_rf(trial):

    rf_params = {
            "max_features": trial.suggest_float("max_features", 0.3, 1),
            "max_depth": trial.suggest_int("max_depth", 2, 15),
            "min_samples_leaf": trial.suggest_float("min_samples_leaf", 0.01, 0.5),
            "bootstrap": trial.suggest_categorical("bootstrap", [False, True]),
            "n_estimators": trial.suggest_int("n_estimators", 10, 200)
        }

    model = RandomForestRegressor(random_state=SEED, **rf_params)


    score = -np.mean(
        cross_val_score(
                model,
                X_train,
                y_train,
                cv=KFold(n_splits=5),
                n_jobs=-1,
                scoring='neg_mean_squared_error')
        )

    return score

study_rf = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.HyperbandPruner()

)

study_rf.optimize(objective_rf, n_trials=n_trials)

In [ ]:
# train an XBGBoost model with tpe sampler and median pruner
def objective_xgb(trial):

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 5000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0, log=True),
        "max_delta_step": trial.suggest_int("max_delta_step", 0, 20),
        "reg_lambda": trial.suggest_float("reg_lambda",1e-9, 100.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-9, 100.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 15),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.1, 0.5, log=True),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1e-6, 500, log=True)
    }


    model = XGBRegressor(random_state=SEED, objective="reg:squarederror", **params)

    score = -np.mean(
        cross_val_score(
                model,
                X_train,
                y_train,
                cv=KFold(n_splits=5),
                n_jobs=-1,
                scoring='neg_mean_squared_error')
        )

    return score

study_xgb = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.HyperbandPruner()
)

study_xgb.optimize(objective_xgb, n_trials=n_trials)

In [55]:
best_hps = {
    "DecisionTree": study_dt.best_params,
    "RandomForest": study_rf.best_params,
    "XGBoost": study_xgb.best_params,
}

In [ ]:
best_hps

In [8]:
best_hps = {'DecisionTree': {'max_depth': 14, 'min_samples_leaf': 0.010039169930176879},
            'RandomForest': {'max_features': 0.38554104875151896,
            'max_depth': 10,
            'min_samples_leaf': 0.010168225931922027,
            'bootstrap': False,
            'n_estimators': 124},
            'XGBoost': {'n_estimators': 4860,
            'learning_rate': 0.49003165411341526,
            'max_delta_step': 20,
            'reg_lambda': 1.1231181925328875e-05,
            'reg_alpha': 3.426318577751964e-08,
            'subsample': 0.89607666469783,
            'colsample_bytree': 0.8147384539995441,
            'max_depth': 4,
            'min_child_weight': 10,
            'gamma': 0.1296891015376799,
            'scale_pos_weight': 2.2740058630279575e-05}}

# Fitting the Models with Best HPs

In [9]:

trained_models = {
    "DecisionTree": DecisionTreeRegressor(random_state=SEED,
                                          **best_hps["DecisionTree"]).fit(X_train, y_train),

    "RandomForest": RandomForestRegressor(random_state=SEED,
                                          **best_hps["RandomForest"]).fit(X_train, y_train),


    "XGBoost": XGBRegressor(random_state=SEED,
                            **best_hps["XGBoost"],
                            objective="reg:squarederror").fit(X_train, y_train),

}


In [19]:
pd.DataFrame.from_dict(best_hps, orient="index").T.to_csv("hyperparameters_when_all_features_are_chosen.csv")

# Evaluation

In [10]:
evaluations_dictionary = {
    "DecisionTree_Train": evaluation(trained_models["DecisionTree"],
                                     X_train, y_train),
    "DecisionTree_Test": evaluation(trained_models["DecisionTree"],
                                    X_test, y_test),

    "RandomForest_Train": evaluation(trained_models["RandomForest"],
                                     X_train, y_train),
    "RandomForest_Test": evaluation(trained_models["RandomForest"],
                                    X_test, y_test),

    "XGBoost_Train": evaluation(trained_models["XGBoost"],
                                X_train, y_train),
    "XGBoost_Test": evaluation(trained_models["XGBoost"],
                               X_test, y_test),

}

evaluation_df = pd.DataFrame(evaluations_dictionary)
evaluation_df.T.to_csv("model_evaluation.csv")


In [ ]:
evaluation_df.T

In [22]:
plt.style.use(['science', 'nature', 'high-contrast', "no-latex"])


In [ ]:
for model in trained_models.keys():
    for dataset in [("Train", X_train, y_train), ("Test", X_test, y_test)]:
        data_type, X_data, y_data = dataset
        fig, ax = plt.subplots(figsize=(7, 7))
        PredictionErrorDisplay.from_estimator(
            trained_models[model],
            X_data,
            y_data,
            kind="actual_vs_predicted",
            scatter_kwargs={"alpha": 1,
                            "color": "#004488",  
                            "label": f"{data_type} MAE: {int(evaluation_df.loc['MeanAbsoluteError', f'{model}_{data_type}'].round(0))}", 
                            "s":45},
            line_kwargs={"color": "black"},
            ax=ax)

        ax.set_ylabel(f"Actual {y.name}", fontsize=14, fontweight='bold')  
        ax.set_xlabel(f"Predicted {y.name}", fontsize=14, fontweight='bold')  
        ax.legend(fontsize=9, loc="upper left", fancybox=True, shadow=True, ncol=1, frameon=False)  
        ax.tick_params(axis='x', labelsize=12)  # Increased font size for x-axis numbers
        ax.tick_params(axis='y', labelsize=12)  # Increased font size for y-axis numbers
        
        from matplotlib.ticker import AutoMinorLocator  # Import minor locator
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))  # Add minor ticks to x-axis
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))  # Add minor ticks to y-axis
        
        ax.grid(False)  # Disable gridlines
        plt.tight_layout()
        plt.savefig(f"evaluation actual_vs_predicted_{model}_{data_type}.svg")
        # plt.close()
        # gc.collect()

In [ ]:
for model in trained_models.keys():
    fig, ax = plt.subplots(figsize=(7, 7))
    PredictionErrorDisplay.from_estimator(
        trained_models[model],
        X_test,
        y_test,
        kind="residual_vs_predicted",
        scatter_kwargs={"alpha": 1,
                        "s":40,
                        "color": "#DD3C2D",
                        "label": f"Test R2: {evaluation_df.loc['R2', f'{model}_Test'].round(3)}",
                        },
        line_kwargs={"color": "black"},
        ax=ax)


    PredictionErrorDisplay.from_estimator(
        trained_models[model],
        X_train,
        y_train,
        kind="residual_vs_predicted",
        scatter_kwargs={"alpha": 0.3,
                        "s":40,
                        "color": "#364B9A",
                        "label": f"Train R2: {evaluation_df.loc['R2', f'{model}_Train'].round(3)}",
                        "marker": "X"
                        },
        line_kwargs={"color": "black"},
        ax=ax)



    ax.set_xlabel(f"Predicted {y.name}", fontsize=14)
    ax.set_ylabel(f"Residuals\n(Actual - Predicted)", fontsize=14)
    ax.legend(fontsize=9, loc="lower left", fancybox=True, shadow=True, ncol=1)
    ax.tick_params(axis='x', labelsize=9)
    ax.tick_params(axis='y', labelsize=9)
    plt.tight_layout()
    plt.savefig(f"evaluation_residual_{model}.svg", format='svg', dpi=1000, bbox_inches='tight')
    # plt.close()
    # gc.collect()

In [ ]:
for model in trained_models.keys():
    residuals = y_test - trained_models[model].predict(X_test)
    fig, ax = plt.subplots(figsize=(7, 7))
    sm.qqplot(residuals, line ='q', ax=ax)
    ax.set_ylabel("Sample Quantiles", fontsize=14)
    ax.set_xlabel("Theoretical Quantiles", fontsize=14)
    ax.tick_params(axis='x', labelsize=9)
    ax.tick_params(axis='y', labelsize=9)
    plt.savefig(f"evaluation qqplot_{model}.svg", format='svg', dpi=1000, bbox_inches='tight')
    # plt.close()
    # gc.collect()

In [ ]:
for model in trained_models.keys():
    residuals_test = y_test - trained_models[model].predict(X_test)
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.text(0.63, 0.7,
            f"Mean: {residuals_test.mean().round(3)}\nStd: {residuals_test.std().round(3)}",
            transform=ax.transAxes, fontsize=9)
    sns.kdeplot(residuals_test, ax=ax, label=f"{model}", color="#6EA5CD", linewidth=2)
    ax.set_ylabel("Density", fontsize=14)
    ax.set_xlabel("Error", fontsize=14)
    plt.legend(fontsize=9, loc="upper left", fancybox=True, shadow=True, ncols=1)
    ax.tick_params(axis='x', labelsize=9)
    ax.tick_params(axis='y', labelsize=9)
    plt.savefig(f"Evaluation_kde_residuals_{model}.svg", format='svg', dpi=1000, bbox_inches='tight')
#     plt.close()
#     gc.collect()

## Explanation

## SHAP

In [27]:
plt.rcdefaults()
# plt.style.use()


mpl_global_config = {
    'figure.dpi': 100,
    'savefig.dpi': 1200,
    'savefig.transparent': False,
    'savefig.bbox': 'tight',
    'axes.axisbelow': True,
    'axes.spines.top': True,
    'axes.spines.right': True,
    'axes.spines.bottom': True,
    'axes.spines.left': True,
    "axes.grid": True,
    'grid.linewidth': 0.3,

}


plt.rcParams.update(mpl_global_config)

In [12]:
explainer = shap.TreeExplainer(trained_models["XGBoost"])
shap_values = explainer.shap_values(X_test)
shap_interaction_values = explainer.shap_interaction_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar", color="#DD3C2D", show=False, max_display=30)
plt.gca().spines['top'].set_visible(True)
plt.gca().spines['right'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)
plt.gca().spines['left'].set_visible(True)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.tight_layout()
plt.savefig("Explanation_FeatureImportancePlot.svg", format='svg', dpi=1000, bbox_inches='tight')
# plt.close()
# gc.collect()

In [ ]:
import matplotlib.pyplot as plt
import shap

# Generate the SHAP summary plot
shap.summary_plot(shap_values, X_test, show=False, max_display=30)

# Customize spines (axis lines)
ax = plt.gca()  # Get current axes
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

# Set spine colors and widths
for spine in ax.spines.values():
    spine.set_color('black')  # Set spine color to black
    spine.set_linewidth(2)     # Make spines bolder

# Set title and labels in bold black
plt.title('SHAP Summary Plot', color='black', fontsize=14, fontweight='bold')  # Title in bold black

# Customize tick parameters to bold the numbers
ax.tick_params(axis='both', which='major', labelcolor='black', labelsize=12) 
for tick in ax.get_xticklabels() + ax.get_yticklabels():  # Get all tick labels
    tick.set_fontsize(12)  # Increased font size for tick labels
    tick.set_fontweight('bold')  # Set font weight to bold

# Increase space between x label and axis
ax.set_xlabel('SHAP Value (impact on model output)', labelpad=15, color='black', fontsize=12, fontweight='bold')  # Adjust labelpad as needed

# Save the plot 
plt.savefig("Explanation_SummaryImportancePlot_2.svg", format='svg', dpi=3000, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import shap

for main_feature in list(X_test.columns):
    fig, ax = plt.subplots(figsize=(7, 7))  
    
    # Generate the SHAP dependence plot
    shap.dependence_plot(
        (main_feature, main_feature),
        shap_interaction_values, X_test,
        display_features=X_test.iloc[:, :],
        dot_size=50,  # Increase dot size for better visibility
        show=False,
        ax=ax
    )

    # Customize spines (axis lines)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    
    # Set spine colors and widths
    for spine in ax.spines.values():
        spine.set_color('black')  # Set spine color to black
        spine.set_linewidth(2)     # Make spines bolder

    # Set title and labels in bold black
    plt.title(f'SHAP Dependence Plot for {main_feature}', color='black', fontsize=14, fontweight='bold')  # Title in bold black
    plt.xlabel(main_feature, color='black', fontsize=12, fontweight='bold')  # X-axis label in bold black
    plt.ylabel('SHAP Value', color='black', fontsize=12, fontweight='bold')  # Y-axis label in bold black

    # Customize tick parameters to bold the numbers
    ax.tick_params(axis='both', which='major', labelcolor='black', labelsize=10)  # Major ticks in black
    for tick in ax.get_xticklabels() + ax.get_yticklabels():  # Get all tick labels
        tick.set_fontsize(10)  
        tick.set_fontweight('bold')  

    # Manually plot the markers in blue
    scatter = ax.collections[0]  # Get the scatter collection from the dependence plot
    scatter.set_facecolor('blue') 

    # Save the plot 
    main_feature_safe = main_feature.replace("/", "")  # Remove slashes from feature names
    plt.savefig(f"Explanation_Singular_{main_feature_safe}_dependence_plot.svg", format='svg', dpi=3000, bbox_inches='tight')



In [ ]:
# compute SHAP values
shap_values = explainer(X_test[:1000])

shap.plots.heatmap(shap_values, instance_order=shap_values.sum(1))



In [ ]:
shap.plots.scatter(shap_values[:, "Temperature (Degrees Celsius)"], color=shap_values[:, "Alkalinity (mg CaCO3/L)"])


In [13]:
clustering = shap.utils.hclust(X, y)


In [ ]:
shap.plots.bar(shap_values, clustering=clustering, clustering_cutoff=0.8, show=False)

# Customize the ticks and labels for the existing plot
plt.xticks(fontsize=18, fontweight="bold", color="black")  
plt.yticks(fontsize=18, fontweight="bold", color="black")  

# Customize the axis labels
plt.xlabel(plt.gca().get_xlabel(), fontsize=14, fontweight="bold", color="black")  
plt.ylabel(plt.gca().get_ylabel(), fontsize=14, fontweight="bold", color="black")  


plt.savefig("bar_plot.svg", format='svg', dpi=3000, bbox_inches='tight')

# Display the updated plot
plt.show()


In [29]:
# download results
import os
from zipfile import ZipFile

with ZipFile("TreeBasedModels.zip", "w") as z:
    for file in os.listdir():
        if file.endswith((".svg", ".csv")):
            z.write(file)